<a href="https://colab.research.google.com/github/notmanan/Depression-Detection-Through-Multi-Modal-Data/blob/master/CNN__Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connecting Drive to Google Colab**

In [0]:
from google.colab import drive
drive.mount('/content/drive')
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from keras import metrics
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dense

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Using TensorFlow backend.


In [0]:
def processData(data):
    X = data.iloc[:,:].values
    X = np.delete(X, 0, 1)
    X = np.delete(X, 1, 1)
    for i in range(len(X)):
        if(isinstance(X[i][5],str) or isinstance(X[i][7],str)):
            X[i] = np.zeros((1, X.shape[1]))
            # print("se" , end = " ")
    return X

def getData(filename):
    data = pd.read_csv(filename,delimiter=',', engine='python')
    X = processData(data)
    return X

def makeDataPoint(patientNo):
    file1 = (patientNo)+"_CLNF_AUs.txt"
    file2 = (patientNo)+"_CLNF_features.txt"
    file3 = (patientNo)+"_CLNF_features3D.txt"
    file4 = (patientNo)+"_CLNF_gaze.txt"
    file5 = (patientNo)+"_CLNF_hog.txt"
    file6 = (patientNo)+"_CLNF_pose.txt"

    X1 = getData(file1)
    X2 = getData(file2)
    X3 = getData(file3)
    X4 = getData(file4)
    X6 = getData(file6)

    X = np.concatenate((X1, X2, X3, X4, X6), 1)
    return X

def scale_down(X):
  X_new = []
  size = 2
  for i in range(int(X.shape[0]/size)):
    cur_row = X[i*size]
    for j in range(1,size):
      if(i+j < X.shape[0]):
        cur_row += X[i+j]
    cur_row = cur_row/size
    X_new.append(cur_row)
  X_new = np.array(X_new)
  return X_new

def decrease_size(X):
  size = 10000
  if(X.shape[0] < size):
    dif = size - X.shape[0] 
    temp = np.zeros((dif,X.shape[1]))
    X = np.concatenate((X,temp),axis = 0)
  elif(X.shape[0] > size):
    X = X[:10000, :]
  return X

def makeDataset(location, folder):
    file  = np.array(pd.read_csv(location,delimiter=',',encoding='utf-8'))[:, 0:2]
    X_temp = []
    Y_temp = []
    for i in range(len(file)):
      patientID = (str(int(file[i][0])))
      string = '/content/drive/My Drive/MCA Dataset/' + folder +"/"+ patientID
      XT = makeDataPoint(string)
      XT = scale_down(XT)
      XT = decrease_size(XT)
      X_temp.append(XT)
      Y_temp.append(int(file[i][1]))
    Y_temp = np.asarray(Y_temp)
    return X_temp, Y_temp

class CNN_Video:
  def __init__(self):
    # Initialising the CNN
    classifier = Sequential()
    # Step 1 - Convolution
    classifier.add(Conv1D(300, 3, input_shape = (10000, 388), activation = 'relu'))
    # Step 2 - Pooling
    classifier.add(MaxPooling1D(pool_size = 2))
    classifier.add(Conv1D(150, 3, activation = 'relu'))
    classifier.add(MaxPooling1D(pool_size = 2))
    # Adding a second convolutional layer
    classifier.add(Conv1D(75, 3, activation = 'relu'))
    classifier.add(MaxPooling1D(pool_size = 2))
    classifier.add(Conv1D(32, 3, activation = 'relu'))
    classifier.add(MaxPooling1D(pool_size = 2))
    # Step 3 - Flattening
    classifier.add(Flatten())
    # Step 4 - Full connection
    classifier.add(Dense(units = 128, activation = 'relu'))
    classifier.add(Dense(units = 1, activation = 'sigmoid'))

  # Compiling the CNN
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy')
    class_weight = { 0: 0.3, 1:0.7}
    self.classifier = classifier

  def fitModel(self, XtrainTotal, YtrainTotal, epoch = 5):
    return self.classifier.fit(XtrainTotal, YtrainTotal, epochs=epoch)

  def predictModel(self, X_test):
    return self.classifier.predict(np.asarray(X_test))


In [0]:
# # Get Dataset
# X_train, Y_train = makeDataset('/content/drive/My Drive/MCA Dataset/train_split_Depression_AVEC2017.csv', 'train_data')
# X_dev, Y_dev = makeDataset('/content/drive/My Drive/MCA Dataset/dev_split_Depression_AVEC2017.csv', 'dev_data')
# X_test, Y_test = makeDataset('/content/drive/My Drive/MCA Dataset/full_test_split.csv', 'test_data')
# YtrainTotal = np.concatenate((Y_train, Y_dev))
# XtrainTotal = X_train + X_dev 
# XtrainTotal = np.asarray(XtrainTotal, dtype=np.float32)

# # Run Model On Test Set
model = CNN_Video()
model.fitModel(XtrainTotal, YtrainTotal, 5)
Y_pred = model.predictModel(X_test)
print(classification_report(Y_test,Y_pred))


Epoch 1/5
4/4 [==============================] - 1s 202ms/step - loss: 1068.7135
Epoch 2/5
4/4 [==============================] - 1s 156ms/step - loss: 0.0000e+00
Epoch 3/5
4/4 [==============================] - 1s 157ms/step - loss: 0.0000e+00
Epoch 4/5
4/4 [==============================] - 1s 157ms/step - loss: 0.0000e+00
Epoch 5/5
4/4 [==============================] - 1s 159ms/step - loss: 0.0000e+00
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

